<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/NLP/Kaggle_upvoted_dataset/Kaggle_NLP_upvoted_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/canggih/voted-kaggle-dataset/notebooks

In [2]:
import pandas as pd
import numpy as np  

In [3]:
data=pd.read_csv('https://github.com/duybluemind1988/Data-science/blob/master/NLP/Kaggle_upvoted_dataset/voted-kaggle-dataset.csv?raw=true')
print(data.shape)
data.head()

(2150, 15)


,Title,Subtitle,Owner,Votes,Versions,Tags,Data Type,Size,License,Views,Download,Kernels,Topics,URL,Description
0,Credit Card Fraud Detection,Anonymized credit card transactions labeled as...,Machine Learning Group - ULB,1241,"Version 2,2016-11-05|Version 1,2016-11-03",crime\nfinance,CSV,144 MB,ODbL,"442,136 views","53,128 downloads","1,782 kernels",26 topics,https://www.kaggle.com/mlg-ulb/creditcardfraud,The datasets contains transactions made by cre...
1,European Soccer Database,"25k+ matches, players & teams attributes for E...",Hugo Mathien,1046,"Version 10,2016-10-24|Version 9,2016-10-24|Ver...",association football\neurope,SQLite,299 MB,ODbL,"396,214 views","46,367 downloads","1,459 kernels",75 topics,https://www.kaggle.com/hugomathien/soccer,The ultimate Soccer database for data analysis...
2,TMDB 5000 Movie Dataset,"Metadata on ~5,000 movies from TMDb",The Movie Database (TMDb),1024,"Version 2,2017-09-28",film,CSV,44 MB,Other,"446,255 views","62,002 downloads","1,394 kernels",46 topics,https://www.kaggle.com/tmdb/tmdb-movie-metadata,Background\nWhat can we say about the success ...
3,Global Terrorism Database,"More than 170,000 terrorist attacks worldwide,...",START Consortium,789,"Version 2,2017-07-19|Version 1,2016-12-08",crime\nterrorism\ninternational relations,CSV,144 MB,Other,"187,877 views","26,309 downloads",608 kernels,11 topics,https://www.kaggle.com/START-UMD/gtd,"Context\nInformation on more than 170,000 Terr..."
4,Bitcoin Historical Data,Bitcoin data at 1-min intervals from select ex...,Zielak,618,"Version 11,2018-01-11|Version 10,2017-11-17|Ve...",history\nfinance,CSV,119 MB,CC4,"146,734 views","16,868 downloads",68 kernels,13 topics,https://www.kaggle.com/mczielinski/bitcoin-his...,Context\nBitcoin is the longest running and mo...


In [6]:
data.Description[0]

"The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.\nIt contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case o

In [7]:
data.Description[1]

'The ultimate Soccer database for data analysis and machine learning\nWhat you get:\n+25,000 matches\n+10,000 players\n11 European Countries with their lead championship\nSeasons 2008 to 2016\nPlayers and Teams\' attributes* sourced from EA Sports\' FIFA video game series, including the weekly updates\nTeam line up with squad formation (X, Y coordinates)\nBetting odds from up to 10 providers\nDetailed match events (goal types, possession, corner, cross, fouls, cards etc...) for +10,000 matches\n*16th Oct 2016: New table containing teams\' attributes from FIFA !\nOriginal Data Source:\nYou can easily find data about soccer matches but they are usually scattered across different websites. A thorough data collection and processing has been done to make your life easier. I must insist that you do not make any commercial use of the data. The data was sourced from:\nhttp://football-data.mx-api.enetscores.com/ : scores, lineup, team formation and events\nhttp://www.football-data.co.uk/ : bett

# Ch06b - Topic Modeling with gensim.ipynb

## Basic Text Wrangling

In [10]:
papers=data.Description
print(len(papers))
papers

2150


0       The datasets contains transactions made by cre...
1       The ultimate Soccer database for data analysis...
2       Background\nWhat can we say about the success ...
3       Context\nInformation on more than 170,000 Terr...
4       Context\nBitcoin is the longest running and mo...
                              ...                        
2145    Context\nFortnite: Battle Royale has over 20 m...
2146    Context\nThis dataset provides the nationaliti...
2147    lem.json\nThis file contains lementized englis...
2148    Context\nThis data set contains weather data f...
2149    Context\nBirths in U.S during 1994 to 2003.\nC...
Name: Description, Length: 2150, dtype: object

In [12]:
papers=papers.astype(str)

In [13]:
%%time
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
  

stop_words = nltk.corpus.stopwords.words('english')
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

def normalize_corpus(papers):
    norm_papers = []
    for paper in papers:
        paper = paper.lower()
        paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
        paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
        paper_tokens = [token for token in paper_tokens if len(token) > 1]
        paper_tokens = [token for token in paper_tokens if token not in stop_words]
        paper_tokens = list(filter(None, paper_tokens))
        if paper_tokens:
            norm_papers.append(paper_tokens)
            
    return norm_papers
    
norm_papers = normalize_corpus(papers)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
CPU times: user 3.14 s, sys: 33.9 ms, total: 3.17 s
Wall time: 3.2 s


In [18]:
print(papers[0])
print(norm_papers[0])

The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.
It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of 

## Text Representation with Feature Engineering

In [ ]:
import gensim

bigram = gensim.models.Phrases(norm_papers, min_count=20, threshold=20, delimiter=b'_') 
# higher threshold fewer phrases.
bigram_model = gensim.models.phrases.Phraser(bigram)

In [20]:
norm_corpus_bigrams = [bigram_model[doc] for doc in norm_papers]
print(norm_corpus_bigrams[0])

['datasets', 'contains', 'transaction', 'made', 'credit', 'card', 'september', 'european', 'cardholder', 'dataset', 'present', 'transaction', 'occurred', 'two', 'day', 'fraud', 'transaction', 'dataset', 'highly', 'unbalanced', 'positive', 'class', 'fraud', 'account', 'transaction', 'contains', 'numerical', 'input', 'variable', 'result', 'pca', 'transformation', 'unfortunately', 'due', 'confidentiality', 'issue', 'cannot', 'provide', 'original', 'feature', 'background', 'information', 'data', 'feature', 'v1', 'v2', 'v28', 'principal', 'component', 'obtained', 'pca', 'feature', 'transformed', 'pca', 'time', 'amount', 'feature', 'time', 'contains', 'second', 'elapsed', 'transaction', 'first', 'transaction', 'dataset', 'feature', 'amount', 'transaction', 'amount', 'feature', 'used', 'example', 'dependant', 'cost', 'senstive', 'learning', 'feature', 'class', 'response', 'variable', 'take', 'value', 'case', 'fraud', 'otherwise', 'given', 'class', 'imbalance', 'ratio', 'recommend', 'measuring

In [21]:
# Create a dictionary representation of the documents.
# Dictionary encapsulates the mapping between normalized words and their integer ids.
dictionary = gensim.corpora.Dictionary(norm_corpus_bigrams)

In [22]:
print('Sample word to number mappings:', list(dictionary.items())[:15])
print('Total Vocabulary Size:', len(dictionary))

Sample word to number mappings: [(0, 'ac'), (1, 'account'), (2, 'accuracy'), (3, 'amount'), (4, 'analysed'), (5, 'andrea'), (6, 'area'), (7, 'artml'), (8, 'auprc'), (9, 'available'), (10, 'background'), (11, 'big'), (12, 'bontempi'), (13, 'brufence'), (14, 'bruxelles')]
Total Vocabulary Size: 32097


In [23]:
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.6)
print('Sample word to number mappings:', list(dictionary.items())[:15])
print('Total Vocabulary Size:', len(dictionary))

Sample word to number mappings: [(0, 'ac'), (1, 'account'), (2, 'accuracy'), (3, 'amount'), (4, 'area'), (5, 'available'), (6, 'background'), (7, 'big'), (8, 'cannot'), (9, 'card'), (10, 'case'), (11, 'class'), (12, 'classification'), (13, 'collected'), (14, 'component')]
Total Vocabulary Size: 1821


In [25]:
# Transforming corpus into bag of words vectors
bow_corpus = [dictionary.doc2bow(text) for text in norm_corpus_bigrams]
print(bow_corpus[0][:50])

[(0, 3), (1, 1), (2, 2), (3, 3), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 3), (12, 2), (13, 1), (14, 1), (15, 1), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 7), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)]


doc2bow: Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

If allow_update is set, then also update dictionary in the process: create ids for new words. At the same time, update document frequencies -- for each word appearing in this document, increase its document frequency (self.dfs) by one.

If allow_update is not set, this function is const, aka read-only.

In [26]:
print([(dictionary[idx] , freq) for idx, freq in bow_corpus[1][:50]])

[('card', 1), ('case', 1), ('class', 2), ('day', 1), ('european', 1), ('feature', 1), ('first', 1), ('ha', 3), ('http', 2), ('machine_learning', 1), ('original', 2), ('probability', 2), ('project', 1), ('time', 2), ('using', 2), ('value', 1), ('able', 2), ('access', 2), ('achieved', 1), ('across', 1), ('adding', 1), ('algorithm', 1), ('also', 3), ('analysis', 1), ('api', 1), ('appears', 1), ('ask', 1), ('asset', 1), ('attribute', 5), ('away', 1), ('base', 1), ('called', 1), ('changed', 1), ('classifier', 1), ('click', 2), ('co', 1), ('collection', 1), ('column', 1), ('com', 3), ('come', 1), ('commercial', 1), ('compare', 1), ('containing', 1), ('coordinate', 1), ('could', 1), ('country', 1), ('crawling', 2), ('cross', 1), ('database', 2), ('design', 1)]


In [27]:
print('Total number of papers:', len(bow_corpus))

Total number of papers: 2150


## Topic Models with Latent Semantic Indexing (LSI)

In [28]:
%%time
TOTAL_TOPICS = 10
lsi_bow = gensim.models.LsiModel(bow_corpus, id2word=dictionary, num_topics=TOTAL_TOPICS,
                                 onepass=True, chunksize=1740, power_iters=1000)

CPU times: user 1min 51s, sys: 47.4 s, total: 2min 38s
Wall time: 1min 21s


In [29]:
for topic_id, topic in lsi_bow.print_topics(num_topics=10, num_words=20):
    print('Topic #'+str(topic_id+1)+':')
    print(topic)
    print()

Topic #1:
0.880*"university" + 0.124*"player" + 0.102*"number" + 0.090*"college" + 0.090*"wa" + 0.088*"file" + 0.087*"year" + 0.087*"time" + 0.079*"team" + 0.075*"csv" + 0.068*"one" + 0.066*"name" + 0.065*"information" + 0.063*"ha" + 0.059*"date" + 0.049*"value" + 0.046*"institute" + 0.045*"university_california" + 0.045*"used" + 0.045*"new"

Topic #2:
-0.451*"university" + 0.295*"player" + 0.206*"number" + 0.185*"team" + 0.177*"time" + 0.176*"year" + 0.172*"wa" + 0.166*"file" + 0.146*"csv" + 0.134*"one" + 0.129*"name" + 0.124*"date" + 0.122*"information" + 0.121*"ha" + 0.101*"goal" + 0.098*"value" + 0.088*"game" + 0.086*"code" + 0.085*"used" + 0.080*"use"

Topic #3:
-0.677*"player" + -0.393*"team" + -0.237*"goal" + -0.152*"zone" + 0.135*"file" + -0.134*"game" + -0.130*"allowed" + 0.112*"csv" + 0.102*"year" + 0.092*"one" + 0.087*"date" + 0.081*"information" + -0.076*"percentage" + 0.073*"name" + 0.072*"value" + -0.068*"individual" + -0.065*"taken" + -0.065*"scored" + -0.060*"relative" 

Let’s take a moment to understand these results. A brief recap on the LSI model— it is based on the principle that words that are used in the same contexts tend to have similar meanings. You can observe in this output that each topic is a combination of terms (which basically tend to convey an overall sense of the topic) and weights. Now the problem here is that we have both positive and negative weights. What does that mean?

Based on existing research and my interpretations, considering we are reducing the dimensionality here to a 10-dimensional space based on the number of topics, the sign on each term indicates a sense of direction or orientation in the vector space for a particular topic. The higher the weight, the more important the contribution. So similar correlated terms have the same sign or direction. Hence, it is perfectly possible for a topic to have two different sub-themes based on the sign or orientation of terms. Let’s separate these terms and try to interpret the topics again.

In [30]:
for n in range(TOTAL_TOPICS):
    print('Topic #'+str(n+1)+':')
    print('='*50)
    d1 = []
    d2 = []
    for term, wt in lsi_bow.show_topic(n, topn=20):
        if wt >= 0:
            d1.append((term, round(wt, 3)))
        else:
            d2.append((term, round(wt, 3)))

    print('Direction 1:', d1)
    print('-'*50)
    print('Direction 2:', d2)
    print('-'*50)
    print()
    

Topic #1:
Direction 1: [('university', 0.88), ('player', 0.124), ('number', 0.102), ('college', 0.09), ('wa', 0.09), ('file', 0.088), ('year', 0.087), ('time', 0.087), ('team', 0.079), ('csv', 0.075), ('one', 0.068), ('name', 0.066), ('information', 0.065), ('ha', 0.063), ('date', 0.059), ('value', 0.049), ('institute', 0.046), ('university_california', 0.045), ('used', 0.045), ('new', 0.045)]
--------------------------------------------------
Direction 2: []
--------------------------------------------------

Topic #2:
Direction 1: [('player', 0.295), ('number', 0.206), ('team', 0.185), ('time', 0.177), ('year', 0.176), ('wa', 0.172), ('file', 0.166), ('csv', 0.146), ('one', 0.134), ('name', 0.129), ('date', 0.124), ('information', 0.122), ('ha', 0.121), ('goal', 0.101), ('value', 0.098), ('game', 0.088), ('code', 0.086), ('used', 0.085), ('use', 0.08)]
--------------------------------------------------
Direction 2: [('university', -0.451)]
--------------------------------------------

Does this make things better? Well, it’s definitely a lot better than the previous interpretation. Here we can see clear themes of modeling being applied in chips and electronic devices, classification and recognition models, neural models talking about the human brain components like cells, stimuli, neurons, cortical components, and even themes around reinforcement learning! We explore these in detail later in a more structured way. Let’s try to get the three major matrices (U, S, and VT) from our topic model, which uses SVD (based on the foundational concepts mentioned earlier)

In [31]:
term_topic = lsi_bow.projection.u
singular_values = lsi_bow.projection.s
topic_document = (gensim.matutils.corpus2dense(lsi_bow[bow_corpus], len(singular_values)).T / singular_values).T
term_topic.shape, singular_values.shape, topic_document.shape

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:502: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)


((1821, 10), (10,), (10, 2150))

In [32]:
document_topics = pd.DataFrame(np.round(topic_document.T, 3), 
                               columns=['T'+str(i) for i in range(1, TOTAL_TOPICS+1)])
print(document_topics.shape)
document_topics.head(15)

(2150, 10)


,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
0,0.006,0.012,0.009,-0.002,0.001,0.002,-0.000,0.014,-0.003,0.001
1,0.020,0.043,-0.041,-0.002,0.015,0.007,0.004,0.027,0.010,-0.006
2,0.014,0.027,0.018,-0.007,0.007,0.012,0.000,0.024,0.021,-0.001
3,0.032,0.035,0.023,-0.016,0.000,0.004,0.005,0.044,0.008,0.003
4,0.007,0.013,0.008,-0.004,0.019,-0.009,-0.008,-0.009,-0.006,0.003
5,0.016,0.032,0.020,-0.002,0.041,-0.011,-0.015,-0.010,-0.002,0.015
6,0.002,0.004,0.003,-0.001,0.001,0.000,0.000,0.007,0.002,0.002
7,0.002,0.004,0.003,-0.000,0.001,0.000,0.000,0.003,0.004,0.000
8,0.009,0.017,0.012,-0.009,0.016,-0.011,-0.009,-0.010,0.005,0.006
9,0.009,0.017,0.007,0.006,0.004,0.002,0.000,0.015,-0.007,-0.003


In [43]:
print(lsi_bow.show_topic(1, topn=20))


[('university', -0.450553573610091), ('player', 0.29506700736267866), ('number', 0.20621154666702776), ('team', 0.18479887219503696), ('time', 0.1766891337550597), ('year', 0.17606283480334114), ('wa', 0.17224652417513153), ('file', 0.16641327395046698), ('csv', 0.14590255454119186), ('one', 0.13429132222503803), ('name', 0.12889782562571633), ('date', 0.12406195789189757), ('information', 0.12192957273946657), ('ha', 0.12087212806902431), ('goal', 0.10073343681324058), ('value', 0.0984630413860938), ('game', 0.08791427352298868), ('code', 0.08565537960353707), ('used', 0.08541151909640803), ('use', 0.08043511731207907)]
[('csv', 0.6334792711458185), ('date', -0.29246379413739515), ('year', -0.22142714981716535), ('number', -0.19452716386112245), ('integer', 0.18294111118787237), ('registration', -0.16034625622593435), ('zero', -0.15748096668186842), ('time', -0.14429339858005663), ('start', -0.134933511373109), ('application', -0.12984797663563105), ('position', -0.11668209010919899),

In [41]:
int(top_topics[0][1])

4

In [45]:
document_numbers = [1, 5, 10]

for document_number in document_numbers:
    top_topics = list(document_topics.columns[np.argsort(-np.absolute(document_topics.iloc[document_number].values))[:3]])
    print('Document #'+str(document_number)+':')
    print('Dominant Topics (top 3):', top_topics)
    print('Paper Summary:')
    print(papers[document_number][:500])
    print('Topic model '+top_topics[0][1]+':',lsi_bow.show_topic(int(top_topics[0][1]), topn=20))
    print()

Document #1:
Dominant Topics (top 3): ['T2', 'T3', 'T8']
Paper Summary:
The ultimate Soccer database for data analysis and machine learning
What you get:
+25,000 matches
+10,000 players
11 European Countries with their lead championship
Seasons 2008 to 2016
Players and Teams' attributes* sourced from EA Sports' FIFA video game series, including the weekly updates
Team line up with squad formation (X, Y coordinates)
Betting odds from up to 10 providers
Detailed match events (goal types, possession, corner, cross, fouls, cards etc...) for +10,000 matches
*16th Oct 201
Topic model 2: [('player', -0.6771529757489971), ('team', -0.39275984613802317), ('goal', -0.2365538810942563), ('zone', -0.15157248883497101), ('file', 0.13549199029909362), ('game', -0.13360002518378492), ('allowed', -0.1296533620537427), ('csv', 0.11207272268683074), ('year', 0.10167480240330877), ('one', 0.09169777743903763), ('date', 0.0873547278113432), ('information', 0.08116850624413084), ('percentage', -0.076383035